## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [7]:
import pandas as pd
solvency = pd.read_csv('/datasets/data.csv')
print('Общая информация')
print('----------------')
solvency.info()
print()
print('Количество уникальных значений столбца children')
print('-----------------------------------------------')
print(solvency['children'].value_counts())
print()
print('Характеристики столбца days_employed')
print('------------------------------------')
print('Минимальное значение',solvency['days_employed'].min())
print('Максимальное значение',solvency['days_employed'].max())
print('Количество значений больше ноля',solvency[solvency['days_employed'] > 0]['children'].count())
print('Количество значений меньше ноля',solvency[solvency['days_employed'] < 0]['children'].count())
print('Количество пропущенных значений',solvency[solvency['days_employed'].isnull()]['children'].count())
print()
print('Уникальные значения столбца dob_years')
print('-------------------------------------')
print(solvency['dob_years'].unique())
print('Количество нулевых значений',solvency[solvency['dob_years'] == 0]['children'].count())
print()
print('Уникальные значения столбца education')
print('------------------------------------')
print(solvency['education'].unique())
print()
print('Количество уникальных значений столбца education_id')
print('---------------------------------------------------')
print(solvency['education_id'].value_counts())
print()
print('Уникальные значения столбца family_status')
print('-----------------------------------------')
print(solvency['family_status'].unique())
print()
print('Количество уникальных значений столбца family_status_id')
print('-------------------------------------------------------')
print(solvency['family_status_id'].value_counts())
print()
print('Количество уникальных значений столбца gender')
print('---------------------------------------------')
print(solvency['gender'].value_counts())
print(solvency[solvency['gender'] == 'XNA'])
print()
print('Информация по столбцу income_type')
print('---------------------------------')
print('Статистика уникальных значений')
print(solvency['income_type'].value_counts(normalize=True))
print()
print('Ежемесячный доход безработного')
print(solvency[solvency['income_type'] == 'безработный']['total_income'])
print()
print('Ежемесячный доход студента')
print(solvency[solvency['income_type'] == 'студент']['total_income'])
print()
print('Ежемесячный доход в декрете')
print(solvency[solvency['income_type'] == 'в декрете']['total_income'])
print()
print('Статистика уникальных значений с не указанным уровнем дохода')
print(solvency[solvency['total_income'].isna()]['income_type'].value_counts(normalize=True))
print()
print('Статистика уникальных значений в строках являющихся дубликатами')
print(solvency[solvency.duplicated()]['income_type'].value_counts(normalize=True))
print()
print('Количество уникальных значений столбца debt')
print('-------------------------------------------')
print(solvency['debt'].value_counts())
print()
print('Характеристики столбца total_income')
print('-------------------------------')
print('Минимальное значение',solvency['total_income'].min())
print('Максимальное значение',solvency['total_income'].max())
print('Количество пропущенных значений в столбцах days_employed и total_income',
      solvency[(solvency['days_employed'].isnull()) & (solvency['total_income'].isnull())]['children'].count())
print()
print('Количество уникальных значений столбца purpose')
print('----------------------------------------------')
print(solvency['purpose'].value_counts())
print()
print('Количество истинных дубликатов')
print('------------------------------')
print(solvency.duplicated().sum())

Общая информация
----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Количество уникальных значений столбца children
-----------------------------------------------
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Характеристики столбца days_employed
--------------------------

### Вывод

Каждая строка таблицы содержит информацию о заемщиках банка и целях их кредита.
Проблемы, которые нужно решать: пропуски данных, а так же истинные и смысловые дубликаты значений. Для проверки рабочих гипотез особенно ценны столбцы 'children', 'total_income', family_status, 'income_type', 'debt' и 'purpose'.

Предоставленные данные требуют предобработки. Наблюдаются следующие проблемы:
1. В столбце 'children' присутствуют артефакты в виде отрицательного значения "-1" и несоизмеримо большого "20". 
Ошибка ввода в виде "-1" могла появиться при машинном распознавании не корректно заполненных анкетных данных представленных в бумажном виде. И наиболее вероятно соответствует значению "1".
Ошибка "20" вероятно связана с неаккуратностью операциониста вводящего анкетные данные в базу и связана с одновременным нажатием цифр "2" и "0" расположенных рядом на NumPad'е, хотя при этом должно наблюдаться соответствующее количество ошибок "10". Для устранения данной ошибки следует заменить значение 20 на 2.
2. В столбцах 'days_employed' и 'total_income' имеются пропущенные значения, при чём значения пропущены одновременно в обоих столбцах. 
Вероятно заявитель предпочёл не указывать данную информацию.
3. В столбце 'days_employed' в основном отрицательные значения, что противоречит смыслу данной позиции, так как стаж не может быть отрицательным;
Большое количество отрицательных значений вероятно характеризует схему хранения данных в разных базах.
Для формирования отчета на поставленные вопросы значения данного столбца не важны.
4. Значения в столбцах 'days_employed' и 'total_income' сложны для восприятия, что связано с длинной этих значений.
Следует привести эти данные к целочисленному виду
5. В столбце 'dob_years' присутствуют нулевые значения.
По-видимому данные значения носят случайный характер. Как рабочий вариант: ошибки могли возникнуть при ручном вводе данных, когда не пропечатывается первая цифра.
Для формирования отчета на поставленные вопросы значения данного столбца не важны.
6. В столбце 'education' значения имеют различное форматирование: строчные и прописные буквы. 
Данные ошибки так же носят случайный характер и могут быть обоснованы привычками ввода данных разними операционистами.
7. В столбце 'family_status' присутствуют значения 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем' что,  не имеет юридических отличий от значения 'Не женат / не замужем' и имеет смысл их объединить. Значение 'Не женат / не замужем' имеет прописные буквы, что так же создает сложности при анализе.
8. В столбце 'gender' присутствует артефакт в виде значения 'XNA'. 
Происхождение данной ошибки повидимому случайно. При этом никаких отличительных признаков данного клиента в базе нет.
Можно только предположить, что с вероятностью примерно 60 - 70% это женщина, т.к. именно такое количество клиентов женского пола в базе. 
9. В столбце 'income_type' присутствуют единичные "артефакты": предприниматель, безработный, студент, в декрете.
Возникновение артефактов может быть связано с отсутствием утвержденного шаблона типов занятости. А так же с не желеанием клиентов раскрывать собственный тип занятости, о чем говорит большое количество нейтральных 'сотрудников'.
Сравнивая статистику типов занятости клиентов в общей таблице и среди клиентов с пропущенными значениями дохода, получаем, что они вточности совпадают. Вероятно мы имеем дело именно с процентом людей (не зависящим от типа занятости), которые не хотят указывать данную информацию.
В целом для решения поставленных задач данные этого столбца необходимы для заполнения пропусков в ежемесячном доходе клиентов.
10. Смысловые значения в столбце 'purpose' часто повторяются.
По-видимому при сборе этих данных не был установлен шаблон ввода.
11. База данных содержит дубликаты данных. На данный момент их 54. Однако их количество может вырасти если понизить регистр столбца "education".
Возникновение таких дубликатов может быть связано как с ошибками работы операциониста, так и с неаккуратным объединением различных баз данных.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [8]:
#Перед обработкой пропусков в столбце 'total_income' избавимся от истинных дубликатов.
print('Количество истинных дубликатов в начале обработки',solvency.duplicated().sum())
print()
#Для начала понизим регистр столбца "education".
solvency['education'] = solvency['education'].str.lower()
#Проверим соответствуют ли значения столбца 'education' с пониженным регистром, значениям столбца 'education_id'
print('Количество уникальных значений столбца education_id')
print('---------------------------------------------------')
print(solvency['education_id'].value_counts())
print()
print('Соответствие группы "высшее" образование идентификатору "0" -',
      solvency[(solvency['education'] == 'высшее')&(solvency['education_id'] == 0)]['education_id'].count())
print('Соответствие группы "среднее" образование идентификатору "1" -',
      solvency[(solvency['education'] == 'среднее')&(solvency['education_id'] == 1)]['education_id'].count())
print('Соответствие группы "неоконченное высшее" образование идентификатору "2" -',
      solvency[(solvency['education'] == 'неоконченное высшее')&(solvency['education_id'] == 2)]['education_id'].count())
print('Соответствие группы "начальное" образование идентификатору "3" -',
      solvency[(solvency['education'] == 'начальное')&(solvency['education_id'] == 3)]['education_id'].count())
print('Соответствие группы "ученая степень" образование идентификатору "4" -',
      solvency[(solvency['education'] == 'ученая степень')&(solvency['education_id'] == 4)]['education_id'].count())
print('Количество дубликатов после понижения регистра',solvency.duplicated().sum())
print()
#Теперь, когда ошибки форматирования устранены можно удалить истинные дубликаты
solvency_dropdup = solvency.drop_duplicates().reset_index(drop=True)

#Устраним ошибки в столбце "days_employed"
#Устраним отрицательные значения и 
#приведем значения трудового стажа к одним единицам измерения с возрастом клиента, к годам
solvency_dropdup['years_employed'] = solvency_dropdup['days_employed'].abs() / 360
#print(solvency_dropdup[['years_employed','dob_years']].head(20))
#Появились артефакты в виде сотен и тысяч лет трудового стажа
#приведём значения к нормальному виду:
#значения не должны быть больше "возраст - 16" в годах.
#или (если возраст не указан) максимального возраста для данного типа занятости
dob_years_max_on_job = solvency_dropdup.groupby('income_type')['dob_years'].max()
#создадим функцию, которая преобразует значения трудового стажа 
#в часах или минутах, к значениям в годах, если они больше "возраст - 16"
def years_employed(row):
    days_employed = row['days_employed']
    income_type = row['income_type']
    try:
        years_employed = int(abs(days_employed) / 360)
        hours_employed2years_employed = int(years_employed / 24)
        minutes_employed2years_employed = int(hours_employed2years_employed / 60)
    except:
        years_employed = days_employed
        hours_employed2years_employed = days_employed
        minutes_employed2years_employed = days_employed
    dob_years = row['dob_years']
    dob_years_max = dob_years_max_on_job[income_type]
    if dob_years != 0 and years_employed > dob_years - 16:
        if hours_employed2years_employed > dob_years - 16:
            return minutes_employed2years_employed
        return hours_employed2years_employed
    if dob_years == 0 and years_employed > dob_years_max - 16:
        if hours_employed2years_employed > dob_years_max - 16:
            return minutes_employed2years_employed
        return hours_employed2years_employed
    return years_employed
solvency['years_employed'] = solvency.apply(years_employed,axis=1)
solvency = solvency[['children','years_employed','dob_years','education',
                                     'education_id','family_status','family_status_id','gender',
                                     'income_type','debt','total_income','purpose']]
print(solvency[['years_employed','dob_years']].head(20))
print(solvency[solvency['dob_years'] == 0][['years_employed','dob_years']].head(20))
print('Количество дубликатов после замены "days_employed" на "years_employed" ',
      solvency.duplicated().sum())

#Заполним пропущенные значения в столбце 'dob_years'
#Для начала посмотрим для каких типов занятости пропущены значения возраста
#Заполняем медианными значениями в зависимости от стажа и типа занятости клиента
#Если значение стажа пропущено то заполняем медианным значением для типа занятости
print(solvency[solvency['dob_years'] == 0]['income_type'].unique())
worker_dob = solvency[(solvency['income_type'] == 'сотрудник')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
worker_dob_median = solvency[(solvency['income_type'] == 'сотрудник')&(solvency['dob_years'] !=
                                                   0)]['dob_years'].median()
partner_dob = solvency[(solvency['income_type'] == 'компаньон')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
partner_dob_median = solvency[(solvency['income_type'] == 'компаньон')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
retiree_dob = solvency[(solvency['income_type'] == 'пенсионер')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
retiree_dob_median = solvency[(solvency_years_employed['income_type'] == 'пенсионер')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
state_dob = solvency[(solvency['income_type'] == 'госслужащий')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
state_dob_median = solvency[(solvency['income_type'] == 'госслужащий')&(solvency['dob_years'] !=
                                                   0)].groupby('years_employed')['dob_years'].median()
def zero2median(row):
    dob_years = row['dob_years']
    income_type = row['income_type']
    years_employed = row['years_employed']
    try:
        if dob_years == 0:
            if income_type == 'сотрудник':
                zero2median = worker_dob[years_employed]
                return zero2median
            if income_type == 'компаньон':
                zero2median = partner_dob[years_employed]
                return zero2median
            if income_type == 'пенсионер':
                zero2median = retiree_dob[years_employed]
                return zero2median
            if income_type == 'госслужащий':
                zero2median = state_dob[years_employed]
                return zero2median
        return dob_years
    except:
        if dob_years == 0:
            if income_type == 'сотрудник':
                return worker_dob_median
            if income_type == 'компаньон':
                return partner_dob_median
            if income_type == 'пенсионер':
                return retiree_dob_median
            if income_type == 'госслужащий':
                return state_dob_median
        return dob_years
solvency['dob_years'] = solvency.apply(zero2median,axis=1)

#Заполним пропущенные значения в столбце 'total_income'
print('Статистика уникальных типов занятости с не указанным уровнем дохода')
solvency_isna = solvency[solvency['total_income'].isna()]
print(solvency_isna['income_type'].value_counts(normalize=True))
print()
#Статистика по типам занятости с не указанным уровнем дохода
#хорошо совпадает со статистикой в общем массиве данных
#поэтому заменим пропущенные значения ежемесячного дохода медианными для данного типа занятости
#Проверим есть ли зависимость ежемесячного дохода от образования клиента
isna_education = solvency_isna.pivot_table(index=['income_type'], columns='education', values='dob_years', aggfunc='count')
print(isna_education)

#Выделим для нахождения медианных значений ежемесячного дохода необходимые столбцы
solvency_dropna = solvency.dropna()
#Построим сводную таблицу зависимости медианных значений от типа занятости и образования
median_income_on_education = solvency_dropna.pivot_table(index=['income_type'], columns='education', values='total_income', aggfunc='median')
print(median_income_on_education)
#Сравнивая таблицы "isna_education" и "median_income_on_education" видим
#что в пропущенных данных нет уникальных комбинаций образования и типа занятости
#также прослеживается четкая зависимость между уровнем образования и ежемесячным доходом для каждого типа занятости
#Видно, что "безработного с высшим образованием" и "предпренимателя" целесообразно отнести к категории "компаньон"
#А "безработного со средним образованием", "студента" и "в декрете" отнесем к "сотрудникам"
solvency.loc[solvency['income_type'] == 'предприниматель', 'income_type'] = 'компаньон'
solvency.loc[14798, 'income_type'] = 'компаньон'
solvency.loc[3133, 'income_type'] = 'сотрудник'
solvency.loc[solvency['income_type'] == 'студент', 'income_type'] = 'сотрудник'
solvency.loc[solvency['income_type'] == 'в декрете', 'income_type'] = 'сотрудник'

#Заполним пропущенные значения базы данных значениями 'undefined' для последующего удобства обращения с ними
solvency = solvency.fillna('undefined')

#Для заполнения пропущенных значений в столбце 'total_income' создадим специальную функцию
#Функция берет на вход строку датасета. Проверяет ее на наличие пропущенного значения. 
#При наличии пропуска заменяет его медианным значением соответствующим типу занятости клиента.
def na_to_median(row):
    total_income = row['total_income']
    income_type = row['income_type']
    education = row['education']
    median = median_income_on_education.loc[income_type,education]
    if total_income == 'undefined':
        return median
    return total_income

#Применим новую функцию к исходному массиву
solvency['total_income'] = solvency.apply(na_to_median,axis=1)
print()

#Заменим не правильные значения в столбце 'children' в соответствии с положениями из п.п. 1.1
solvency.loc[solvency['children'] == -1,'children'] = 1
solvency.loc[solvency['children'] == 20,'children'] = 2

print('Количество уникальных значений столбца children')
print('-----------------------------------------------')
print(solvency['children'].value_counts())

Количество истинных дубликатов в начале обработки 54

Количество уникальных значений столбца education_id
---------------------------------------------------
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Соответствие группы "высшее" образование идентификатору "0" - 5260
Соответствие группы "среднее" образование идентификатору "1" - 15233
Соответствие группы "неоконченное высшее" образование идентификатору "2" - 744
Соответствие группы "начальное" образование идентификатору "3" - 282
Соответствие группы "ученая степень" образование идентификатору "4" - 6
Количество дубликатов после понижения регистра 71

    years_employed  dob_years
0             23.0         42
1             11.0         36
2             15.0         33
3             11.0         32
4              0.0         53
5              2.0         27
6              7.0         43
7              0.0         50
8             19.0         35
9              6.0         41
10            11

NameError: name 'solvency_years_employed' is not defined

### Вывод

На этапе предобработки в данных обнаружились не только пропуски, но и всяческие виды дубликатов. Их удаление позволило провести анализ точнее. Поскольку сведения о ежемесячном доходе важно сохранить для анализа, заполнили пропущенные значения медианными, соответствующими типу занятости клиентов.
При анализе данных о типе занятости клиентов и их ежемесячном доходе выяснилось,  что тип занятости заполняется хаотически, более половины клиентов склонны скрывать свой тип занятости указывая нейтральное "сотрудник" или не соответствующее действительности "безработный" или "студент".
Так же показано что в каждом типе занятости ежемесячный доход растет с увеличением уровня образования.

### Замена типа данных

In [ ]:
#Изменим тип данных столбца 'total_income' для удобства восприятия на целочисленный
solvency['total_income'] = solvency['total_income'].astype('int')
print(solvency['total_income'].head())

### Вывод

Тип данных столбца 'total_income' был float64, что усложняло восприятие этих данных. Изменили тип данных на целочисленный. 

### Обработка дубликатов

In [ ]:
#В столбце 'family_status' есть совпадающие по смыслу позиции 'гражданский брак' и 'Не женат / не замужем',
#'вдовец / вдова' и 'в разводе'. Имеет смысл объединить совпадающие позиции.

#Предварительно переведем значения в таблице к нижнему регистру
solvency['family_status'] = solvency['family_status'].str.lower()
solvency.loc[solvency['family_status'] == 'гражданский брак','family_status'] = 'не женат / не замужем'
solvency.loc[solvency['family_status'] == 'вдовец / вдова','family_status'] = 'не женат / не замужем'
solvency.loc[solvency['family_status'] == 'в разводе','family_status'] = 'не женат / не замужем'
print(solvency['family_status'].value_counts())

#Заменим так же значения в столбце 'family_status_id'
#Чтобы застраховать себя от возможных не соответсвий между предыдущими значениями и их идентификаторами
#Сформируем этот столбец заново
#Создадим функцию, которая принимает на вход строку датасета и возвращает 
#0 в случае если 'family_status' - 'женат / замужем'
#1 в случае если 'family_status' - 'не женат / не замужем'
def family_status_id(row):
    family_status = row['family_status']
    if family_status == 'не женат / не замужем':
        return 1
    return 0
solvency['family_status_id'] = solvency.apply(family_status_id,axis=1)
print(solvency['family_status_id'].value_counts())

### Вывод

Полученная база данных имела в столбце 'family_status' несколько смысловых дубликатов. При дальнейшем анализе это может привести к не корректным выводам. Поэтому:
1. Заменили значения 'гражданский брак', 'вдовец / вдова' и 'в разводе' на 'не женат / не замужем'
2. Во избежание дубликатов с имевшимся в столбце значением 'Не женат / не замужем' все строки в столбце привели к нижнему регистру
3. Как сопутствующий результат получено, что клиенты связанные узами брака чаще берут кредит в банке, чем клиенты без серьезных отношений.
4. При подготовке данных к заполнению пропусков избавились от истинных дубликатов в базе данных. При чем выявились дополнительные дубликаты после понижения регистра в столбце "education"

### Лемматизация

In [35]:
#Посчитаем сколько различных целей кредита в полученной базе данных
purposes = solvency['purpose'].unique()
print(solvency['purpose'].value_counts())
print('Количество различных целей кредита до стемминга равно',len(solvency['purpose'].unique()))
print()

#Загрузим стеммер
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

#Разберем список целей кредита на основы и посчитаем наиболее часто встречающиеся
stemmed_purposes = []
for purpose in purposes:
    for word in purpose.split():
        stemmed_word = russian_stemmer.stem(word)
        stemmed_purposes.append(stemmed_word)
        
print(pd.Series(stemmed_purposes).value_counts())
print()
            
#Создадим функцию, которая получает на вход строку датасета и возвращает:
#'получение образования' если в 'purpose' была основа 'образован';
#'операции с недвижимостью' если в 'purpose' была основа 'недвижим' или 'жил';
#'проведение свадьбы' если в 'purpose' была основа 'свадьб';
#'покупка автомобиля' если в 'purpose' была основа 'автомобил'.
def stemm_purpose(row):
    purpose = row['purpose']
    for word in purpose.split(' '):
        stemmed_word = russian_stemmer.stem(word)
        if stemmed_word == 'образован':
            return 'получение образования'
        if stemmed_word == 'недвижим' or stemmed_word == 'жил':
            return 'операции с недвижимостью'
        if stemmed_word == 'свадьб':
            return 'проведение свадьбы'
        if stemmed_word == 'автомобил':
            return 'покупка автомобиля'
solvency['purpose'] = solvency.apply(stemm_purpose,axis=1)

#Проверим как изменились цели кредита после стемминга
print(solvency['purpose'].value_counts())
print('Количество различных целей кредита после стемминга равно',len(solvency['purpose'].value_counts()))

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Полученная база данных имела 38 различных целей кредита. При анализе выяснилось что многие цели являются смысловыми дубликатами: cформулированы различным образом, а так же имеют грамматические ошибки.
1. Провели стемминг целей кредита. Наиболее популярными оказались основы слов: "недвижимость", "покупка", "жилье", "образование", "автомобиль".
2. После стемминга столбца 'purpose' количество различных целей кредита сократилось с 38 до 4
3. Проведенный стемминг столбца 'purpose' показал, что около 50% целей получения кредита являются - операции с недвижимостью.

### Категоризация данных

In [36]:
#Анализировать значения дохода клиентов достаточно сложно. Сгруппируем их.
#Создадим из базы данных клиентов выборки одинаковой численности и близким уровнем ежемесячного дохода

total_income_min = solvency['total_income'].min()
total_income_oktile1 = solvency['total_income'].quantile(q=0.125)
total_income_oktile2 = solvency['total_income'].quantile(q=0.250)
total_income_oktile3 = solvency['total_income'].quantile(q=0.375)
total_income_oktile4 = solvency['total_income'].quantile(q=0.5)
total_income_oktile5 = solvency['total_income'].quantile(q=0.625)
total_income_oktile6 = solvency['total_income'].quantile(q=0.75)
total_income_oktile7 = solvency['total_income'].quantile(q=0.875)
total_income_max = solvency['total_income'].max()

#Организуем полученные выборки в таблицу для нотации
columns = ['income_status','income_status_id','income_from','to']
data = [['очень низкий',0,total_income_min,total_income_oktile1],
         ['низкий',1,total_income_oktile1,total_income_oktile2],
         ['ниже медианного',2,total_income_oktile2,total_income_oktile3],
         ['медианный',3,total_income_oktile3,total_income_oktile4],
         ['выше медианного',4,total_income_oktile4,total_income_oktile5],
         ['высокий',5,total_income_oktile5,total_income_oktile6],
         ['очень высокий',6,total_income_oktile6,total_income_oktile7],
         ['гигантский',7,total_income_oktile7,total_income_max]]
total_income_notation = pd.DataFrame(data=data, columns=columns)
print(total_income_notation)
print()

#Создадим функцию принимающую на вход строку датасета и выдающую значения 'income_status' 
#в зависимости от значения 'total_income'
def income_status(row):
    total_income = row['total_income']
    if total_income <= total_income_oktile1:
        return 'очень низкий'
    if total_income > total_income_oktile1 and total_income <= total_income_oktile2:
        return 'низкий'
    if total_income > total_income_oktile2 and total_income <= total_income_oktile3:
        return 'ниже медианного'
    if total_income > total_income_oktile3 and total_income <= total_income_oktile4:
        return 'медианный'
    if total_income > total_income_oktile4 and total_income <= total_income_oktile5:
        return 'выше медианного'
    if total_income > total_income_oktile5 and total_income <= total_income_oktile6:
        return 'высокий'
    if total_income > total_income_oktile6 and total_income <= total_income_oktile7:
        return 'очень высокий'
    return 'гигантский'


#Создадим функцию принимающую на вход строку датасета и выдающую значения 'income_status_id' 
#в зависимости от значения 'total_income'
def income_status_id(row):
    total_income = row['total_income']
    if total_income <= total_income_oktile1:
        return 0
    if total_income > total_income_oktile1 and total_income <= total_income_oktile2:
        return 1
    if total_income > total_income_oktile2 and total_income <= total_income_oktile3:
        return 2
    if total_income > total_income_oktile3 and total_income <= total_income_oktile4:
        return 3
    if total_income > total_income_oktile4 and total_income <= total_income_oktile5:
        return 4
    if total_income > total_income_oktile5 and total_income <= total_income_oktile6:
        return 5
    if total_income > total_income_oktile6 and total_income <= total_income_oktile7:
        return 6
    return 7
solvency['income_status'] = solvency.apply(income_status,axis=1)
solvency['income_status_id'] = solvency.apply(income_status_id,axis=1)
print(solvency[['total_income','income_status','income_status_id']].head(20))

     income_status  income_status_id  income_from         to
0     очень низкий                 0      20667.0    83941.0
1           низкий                 1      83941.0   107719.0
2  ниже медианного                 2     107719.0   127967.5
3        медианный                 3     127967.5   143496.0
4  выше медианного                 4     143496.0   165640.0
5          высокий                 5     165640.0   198149.0
6    очень высокий                 6     198149.0   253994.5
7       гигантский                 7     253994.5  2265604.0

    total_income    income_status  income_status_id
0         253875    очень высокий                 6
1         112080  ниже медианного                 2
2         145885  выше медианного                 4
3         267628       гигантский                 7
4         158616  выше медианного                 4
5         255763       гигантский                 7
6         240525    очень высокий                 6
7         135823        медианный 

### Вывод

В предоставленной базе данных отсутствовали категории для ежемесячного дохода клиентов. Это ограничивало возможности эффективного анализа кредитоспособности клиентов в зависимости от их дохода. 
Провели категоризацию данных о ежемесячном доходе, путем организации восьми групп одинаковой численности, что позволит более точно проанализировать кредитоспосбность клиентов

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
solvency_on_children = solvency[['children','dob_years','debt']]
children_pivot = solvency_on_children.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count').reset_index()
children_pivot['debt_relative'] = children_pivot[1] / (children_pivot[1] + children_pivot[0])
print(children_pivot)

debt  children        0       1  debt_relative
0            0  13086.0  1063.0       0.075129
1            1   4420.0   445.0       0.091470
2            2   1929.0   202.0       0.094791
3            3    303.0    27.0       0.081818
4            4     37.0     4.0       0.097561
5            5      9.0     NaN            NaN


### Вывод

Среди клиентов банка имеющих кредит около 61% не имеют детей. Они так же реже задерживают возврат. Для клиентов с детьми характерно следующее:
1. Вероятность задолженности по возврату кредита на 0.5 - 1.5% выше.
2. Вероятность задолженности по возврату кредита медленно растет с увеличением количества детей у клиента.
3. Однако оба вывода можно делать с осторожностью т.к. может иметь погрешность исследования, например: вероятность задолженности для клиента с тремя детьми и клиента без детей почти одинакова, что довольно сложно объяснить.
Увеличение вероятности задолженности может быть связано с появившиеся в семье дети добавляют неопределенности в планировании трат и возможности семьи к возврату кредита. Что особенно явно отображено в резком росте вероятности между клиентами без детей и клиентами, имеющими одного ребенка.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
solvency_on_family = solvency[['family_status','dob_years','debt']]
family_pivot = solvency_on_family.pivot_table(index=['family_status'], columns='debt', values='dob_years', aggfunc='count').reset_index()
family_pivot['relative'] = family_pivot[1] / (family_pivot[1] + family_pivot[0])
print(family_pivot)

debt          family_status      0    1  relative
0           женат / замужем  11449  931  0.075202
1     не женат / не замужем   8335  810  0.088573


### Вывод

Среди свободных клиентов вероятность задолженности по возврату кредита выше на 1.3%, чем у клиентов связанных узами брака. Здесь вероятно играет тот факт, что супруги отдают кредит совместными усилиями, если у одного возникают финансовые пролемы второй всегда поддержит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
solvency_on_total_income = solvency[['income_status','income_status_id','dob_years','debt']]
total_income_pivot = solvency_on_total_income.pivot_table(index=['income_status','income_status_id'], columns='debt', values='dob_years', aggfunc='count').reset_index()
total_income_pivot['debt_relative'] = total_income_pivot[1] / (total_income_pivot[1] + total_income_pivot[0])
print(total_income_pivot.sort_values('income_status_id'))
print()
print('Нотация для сводной таблицы')
print(total_income_notation)

debt    income_status  income_status_id     0    1  debt_relative
7        очень низкий                 0  2485  206       0.076551
5              низкий                 1  2470  221       0.082126
4     ниже медианного                 2  2456  234       0.086989
3           медианный                 3  2442  249       0.092531
1     выше медианного                 4  2624  235       0.082197
0             высокий                 5  2301  221       0.087629
6       очень высокий                 6  2501  189       0.070260
2          гигантский                 7  2505  186       0.069119

Нотация для сводной таблицы
     income_status  income_status_id  income_from         to
0     очень низкий                 0      20667.0    83941.0
1           низкий                 1      83941.0   107719.0
2  ниже медианного                 2     107719.0   127967.5
3        медианный                 3     127967.5   143496.0
4  выше медианного                 4     143496.0   165640.0
5          

### Вывод

Анализ зависимости вероятности задержки возврата кредита от ежемесячного дохода клиентов показал, что зависимость имеет максимум для клиентов с ежемесячным доходом от 128 до 143 тыс. Повидимому клиенты с "очень низким" доходом более тщательно оценивают свои возможности когда обращаются в банк за кредитом. А клиенты с "гигантским" доходом реже испытывают финансовые проблемы.
В ходе анализа отмечено что: принципы заполнения пропусков в столбце 'total_income', а также наличие дубликатов оказывают существенное влияние на результат: максимум кривой задержки возврата кредита сдвигается в соседний сегмент диаграммы. 

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
solvency_on_purpose = solvency[['purpose','dob_years','debt']]
purpose_pivot = solvency_on_purpose.pivot_table(index=['purpose'], columns='debt', values='dob_years', aggfunc='count').reset_index()
purpose_pivot['debt_relative'] = purpose_pivot[1] / (purpose_pivot[1] + purpose_pivot[0])
print(purpose_pivot)

debt                   purpose      0    1  debt_relative
0     операции с недвижимостью  10058  782       0.072140
1           покупка автомобиля   3478  359       0.093563
2        получение образования   3652  370       0.091994
3           проведение свадьбы   2162  186       0.079216


### Вывод

Анализ четырех целей кредита показал, что:
1. наибольшей вероятностью задержки возврата кредита являются "покупка автомобиля" и "получение образования".
2. "операции с недвижимостью" имеют наименьшую вероятность задержки возврата кредита.
Повидимому "покупка квартиры" как наиболее затратная цель получения кредита заставляет клиентов наиболее серьезно оценивать свои возможности по выплате.

### Шаг 4. Общий вывод

Кредитоспособность клиентов - величина зависящая от многих факторов. Здесь прослеживается основной тренд: чем серьезнее цель получения кредита и ответственнее клиент подходит к решению его получить, тем меньше вероятность задержки возврата. 
Неожиданный вывод: клиенты с ежемесячным доходом близким к среднему (среди подавших заявку на получение кредита) чаще других задерживают его возврат.
Так же, появление ребенка в семье увеличивает вероятность задержки возврата.